## Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import 'Tensorflow' pakage
import tensorflow as tf
from tensorflow import keras

# Check the version of tensorflow
print(tf.__version__)

In [ ]:
# Check if a GPU(in Google server) is allocated
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

In [ ]:
# Acess to google drive
from google.colab import drive
drive.mount('/content/drive')

## Load the SELECTED (Top 30) Feature Dataset
* Results of ML3-1 and ML3-2

In [ ]:
FeatureSelected = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SavedFiles/FeatureSelected.csv', header=None)
FeatureSelected = FeatureSelected.T
FeatureSelected.shape

In [ ]:
# Standardize feature values
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

FeatureSelected_std = StandardScaler().fit_transform(FeatureSelected)
FeatureSelected_std.shape

## Split Training & Test Data
- Use 'train_test_split' function
- It randomly samples the training and testing data according to the designated ratio.

In [ ]:
# Number of data for each condition: 180
NoOfData   = int(FeatureSelected_std.shape[0]/2)

NormalSet   = FeatureSelected_std[:NoOfData , :]
AbnormalSet = FeatureSelected_std[NoOfData: , :]

NormalSet.shape, AbnormalSet.shape

In [ ]:
from sklearn.model_selection    import train_test_split

# Designate test data ratio
TestData_Ratio = 0.2 

TrainData_Nor, TestData_Nor = train_test_split(NormalSet  , test_size=TestData_Ratio, random_state=777)
TrainData_Abn, TestData_Abn = train_test_split(AbnormalSet, test_size=TestData_Ratio, random_state=777)

print(TrainData_Nor.shape, TestData_Nor.shape)
print(TrainData_Abn.shape, TestData_Abn.shape)

## Data Labling (One-hot Encoding)
- Use 'np.zeros' and 'np.ones'
- '[1,0]' refers to 'Normal' and '[1,0]' refers to 'Abnormal' in this tutorial

In [ ]:
TrainLabel_Nor = np.zeros((TrainData_Nor.shape[0],2))
TrainLabel_Abn = np.ones( (TrainData_Abn.shape[0],2)) 
TestLabel_Nor  = np.zeros((TestData_Nor.shape[0],2))
TestLabel_Abn  = np.ones( (TestData_Abn.shape[0],2)) 

TrainLabel_Nor[:,0] = 1  # [1,0]: Normal
TrainLabel_Abn[:,0] = 0  # [0,1]: Abnormal
TestLabel_Nor[:,0]  = 1  # [1,0]: Normal
TestLabel_Abn[:,0]  = 0  # [0,1]: Abnormal

print(TrainLabel_Nor.shape, TestLabel_Nor.shape)
print(TrainLabel_Abn.shape, TestLabel_Abn.shape)

In [ ]:
TestLabel_Nor

## Data and Label Preparation

In [ ]:
TrainData  = np.concatenate([TrainData_Nor , TrainData_Abn ], axis=0)
TestData   = np.concatenate([TestData_Nor  , TestData_Abn  ], axis=0)
TrainLabel = np.concatenate([TrainLabel_Nor, TrainLabel_Abn], axis=0)
TestLabel  = np.concatenate([TestLabel_Nor , TestLabel_Abn ], axis=0)

print(TrainData.shape,  TestData.shape)
print(TrainLabel.shape, TestLabel.shape)

.

.

.

.

.

## Setting hyperparameters for training ANN(Artificial Neural Network) 

In [ ]:
learningRate  = 0.0001
noOfNeuron    = 16
Epoch         = 200

## Designing an ANN architecture (based on Keras)

- Types of Activation Functions: https://keras.io/api/layers/activations/

- Types of Optimization Algorithms: https://keras.io/api/optimizers/

- Types of Loss Functions (for Classification)  https://keras.io/api/losses/probabilistic_losses/

In [ ]:
def ANN_model(input_data):
    keras.backend.clear_session() # clearing the Keras backend session (initiating variables)

    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape = (input_data.shape[1],) ))                                      # Input  Layer
    model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,    name = 'Hidden1'))    # Hidden Layer 1
    model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu,    name = 'Hidden2'))    # Hidden Layer 2
    model.add(keras.layers.Dense(units = 2,          activation = keras.activations.softmax, name = 'Output'))     # Output Layer
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = learningRate), # Optimization algorithm
                  loss = keras.losses.CategoricalCrossentropy(),                   # Loss function (objective function of Optimization)
                  metrics = ['accuracy'])                                          # Metrics to measure during the training process
    return model

In [ ]:
# Check the model architecture and the number of parameters
AnnModel = ANN_model(TrainData)
AnnModel.summary()

In [ ]:
# Check the parameter shape for each layer
for i in range(len(AnnModel.get_weights())):
    print(AnnModel.get_weights()[i].shape)

## ANN Model Training

In [ ]:
tf.random.set_seed(777) # Not necessarily required

# Model traning and validation
TraingHistory  = AnnModel.fit(TrainData, TrainLabel, epochs=Epoch, verbose = 1)

In [ ]:
# Evaluation result for test data (not trained)
Loss, Accuracy = AnnModel.evaluate(TestData,  TestLabel, verbose=0)
Loss, Accuracy # The closer the Loss is to 0 and the closer the accuracy is to 1 (100%), the better.

In [ ]:
# Check the training process (Loss, Accuracy)

fig, loss_ax = plt.subplots(figsize=(8,6))
acc_ax = loss_ax.twinx()

loss_ax.plot(TraingHistory.history['loss'], label='train loss', c = 'tab:red')
loss_ax.set_xlabel('epoch', fontsize=15)
loss_ax.set_ylabel('loss', fontsize=15)
loss_ax.legend(loc='upper left', fontsize=15)

acc_ax.plot(TraingHistory.history['accuracy'], label='train acc', c = 'tab:blue')
acc_ax.set_ylabel('accuracy', fontsize=15)
acc_ax.legend(loc='lower left', fontsize=15)

plt.show()

### [Tip] Defining a callback function to check the training progress at desired epochs

In [ ]:
# Define the callback function
EpochForPrint = 20

class CheckProcess(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keras.callbacks.Callback()
        if epoch%EpochForPrint == 0:
            print("{} Epochs Train Acc. : {:.2f}%  ".format(epoch, logs["accuracy"]*100))

In [ ]:
AnnModel_2 = ANN_model(TrainData)
hist = AnnModel_2.fit(TrainData, TrainLabel, epochs=Epoch, verbose=0, callbacks=[CheckProcess()])

print('Final Train Accuracy : {:.2f}%'.format(hist.history['accuracy'][-1]*100))

Save ML model (ANN) as a file

In [ ]:
# Unlike SVM or KNN, no 'Joblib' package is needed.

AnnModel.save('/content/drive/MyDrive/Colab Notebooks/SavedFiles/ML_Models/ANN_model.h5')

Load the saved ML model (ANN) and test

In [ ]:
LoadedModel = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/SavedFiles/ML_Models/ANN_model.h5')

Loss, Accuracy = LoadedModel.evaluate(TestData, TestLabel, verbose=0)
print('[Performance of ANN model] \n')
print('Accuracy : {:.2f}%'.format(Accuracy*100))

In [ ]:
# Predicted result
Predicted = LoadedModel.predict(TestData)
pd.DataFrame(Predicted)